# Duet AI & Data Analysis

Provide Duet AI with context in the form of a commented statement and watch as a suggestion is provided.

Using the pandas and matplotlib libraries, import and analyze data.

>NOTE: You must import the CSV file to the runtime file system (i.e. drag it in to the root folder using the Files browser.



In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

sns.set_style('darkgrid')

## Import

- Create a pandas dataframe using a `.csv` file and view the header of the file. Ensure that the encoding is `ISO-8859-1` and that the dtype for UPC_Code and Order_Number are set as strings

In [ ]:
# Import Data with specified data types for UPC_Code and Order_Number
receipts_df = pd.read_csv('receipt_data.csv', encoding='ISO-8859-1',
                          dtype={'UPC_Code': str, 'Order_Number': str})
receipts_df.head()

## View
- Output information about the receipts_df dataframe
- Show count of UPC_Codes
- Show unique Item_Details
- Show count of unique Item_Details

**Duet AI** should have generated the following:
- `receipts_df.info()`
- `receipts_df['UPC_Code'].nunique()`
- `receipts_df['Item Details'].nunique()`
- `receipts_df['Item Details'].unique()`

## Modify
- Modify the data so that the Item Details is now Item_Details
- Show count of unique Item_Details
- Convert Order_Date from string to date with the correct format 'MM/DD/YY' and display the dataframe

**Duet AI** should have generated the following:
- `receipts_df.rename(columns={'Item Details': 'Item_Details'}, inplace=True)`
- `receipts_df['Item_Details'].nunique()`
- `receipts_df['Order_Date'] = pd.to_datetime(receipts_df['Order_Date'], format='%m/%d/%y')
receipts_df.head()`

## Examine

- Describe the receipts_df dataframe with `.T`
- Display items with an amount greater than 4.64
- Count of Item_Details
- Show UPC Code that Have Been Purchased More Than Once
- Calculate the average Total Savings for all orders and display as dollars

**Duet AI** Should Have Generated the Following
- `receipts_df.describe().T`
- `receipts_df[receipts_df['Amount'] > 4.64]`
- `receipts_df['Item_Details'].value_counts()`
- `receipts_df['UPC_Code'].value_counts()[receipts_df['UPC_Code'].value_counts() > 1]`
- `average_total_savings = receipts_df['Total_Savings'].mean().round(2)
formatted_average = f"${average_total_savings}"
print(formatted_average)`

In [ ]:
# Group by Order_Date, then count the Item_Details for each date
order_item_counts = receipts_df.groupby('Order_Date')['Item_Details'].count().reset_index()

# Rename the columns for clarity
order_item_counts.columns = ['Order_Date', 'Item_Count']

# Sort the DataFrame by Order_Date in descending order (most recent date first)
order_item_counts = order_item_counts.sort_values(by='Order_Date', ascending=False)

# Display the DataFrame
order_item_counts

In [ ]:
# Group by Order_Date and Order_Number, then calculate the mean of Amount
average_amount_by_date_order = receipts_df.groupby(['Order_Date', 'Order_Number'])['Amount'].mean().reset_index()

# Format the amounts to include a dollar sign
average_amount_by_date_order['Formatted_Amount'] = average_amount_by_date_order['Amount'].map('${:,.2f}'.format)

# Sort the DataFrame by Order_Date in descending order (most recent date first)
average_amount_by_date_order = average_amount_by_date_order.sort_values(by='Order_Date', ascending=False)

# Display the DataFrame
average_amount_by_date_order

## Visualize


- Using matplotlib generate code to display a smooth time series graph of each order (using order date) and the Payment Amount

In [ ]:
# Using matplotlib generate code to display a smooth time series graph of each order (using order date) and the Payment Amount
receipts_df.plot(x='Order_Date', y='Payment_Amount', kind='line')

### Item Count Over Time

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Order_Date']
  ys = series['Item_Count']

  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = order_item_counts.sort_values('Order_Date', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Order_Date')
_ = plt.ylabel('Item_Count')

### Item Count Over Time w/ a Trend Line

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import datetime

# Convert Order_Date to datetime format if it's not already
receipts_df['Order_Date'] = pd.to_datetime(receipts_df['Order_Date'])

def _plot_series(series, series_name, series_index=0):
    palette = list(sns.palettes.mpl_palette('Dark2'))
    xs = series['Order_Date']
    ys = series['Item_Count']

    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

    # Fitting a linear trend line
    z = np.polyfit(pd.to_datetime(xs).map(datetime.datetime.toordinal), ys, 1)
    p = np.poly1d(z)
    plt.plot(xs, p(pd.to_datetime(xs).map(datetime.datetime.toordinal)), "--", color=palette[series_index % len(palette)], label=f'{series_name} Trend Line')

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = order_item_counts.sort_values('Order_Date', ascending=True)
_plot_series(df_sorted, 'Item Count')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Order Date')
plt.ylabel('Item Count')
plt.legend()
plt.show()


### Average Item Price By Order Date

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

# Ensure that Order_Date is in datetime format
receipts_df['Order_Date'] = pd.to_datetime(receipts_df['Order_Date'])

# Group by Order_Date and calculate the mean of Amount, then sort by Order_Date
sorted_average_amount_by_date = receipts_df.groupby('Order_Date')['Amount'].mean().sort_index()

# Plotting the time series
fig, ax = plt.subplots()
sorted_average_amount_by_date.plot(ax=ax, kind='bar')

# Format y-axis to show dollar sign and two decimal places
fmt = '${x:,.2f}'
tick = mtick.StrMethodFormatter(fmt)
ax.yaxis.set_major_formatter(tick)

# Set labels
ax.set_xlabel('Order Date')
ax.set_ylabel('Average Amount')
ax.set_title('Average Amount by Order Date (Chronologically)')

# Show the plot
plt.show()


### Payment Amount Over Time w/ a Trend Line

In [ ]:
# Linear Regression

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress

# Ensure that Order_Date is in datetime format
receipts_df['Order_Date'] = pd.to_datetime(receipts_df['Order_Date'])

# Sort by Order_Date
receipts_df_sorted = receipts_df.sort_values('Order_Date')

# Generate a linear regression line
x = np.arange(len(receipts_df_sorted))
y = receipts_df_sorted['Payment_Amount']
slope, intercept, r_value, p_value, std_err = linregress(x, y)
trendline = intercept + slope * x

# Plotting the data and the trendline
plt.figure(figsize=(10, 5))
plt.plot(receipts_df_sorted['Order_Date'], receipts_df_sorted['Payment_Amount'], label='Payment Amount')
plt.plot(receipts_df_sorted['Order_Date'], trendline, color='red', label='Trend Line')

plt.xlabel('Order Date')
plt.ylabel('Payment Amount')
plt.title('Payment Amount Over Time with Trend Line')
plt.legend()
plt.show()


### Cumulative Amount Increase for Top 10 Items Over Time

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Ensure Order_Date is a datetime
receipts_df['Order_Date'] = pd.to_datetime(receipts_df['Order_Date'])

# Identify the top 10 most purchased items based on frequency or total amount
top_items = receipts_df.groupby('Item_Details')['Amount'].sum().nlargest(10).index

# Filter the DataFrame for only these top 10 items
top_10_df = receipts_df[receipts_df['Item_Details'].isin(top_items)]

# Group by Item_Details and Order_Date and sum the Amount
grouped_data = top_10_df.groupby(['Item_Details', 'Order_Date'])['Amount'].sum().reset_index()

# Pivot the data
pivoted_data = grouped_data.pivot(index='Order_Date', columns='Item_Details', values='Amount').fillna(0)

# Calculate the cumulative sum over time for each item
cumulative_data = pivoted_data.cumsum()

# Plotting
plt.figure(figsize=(15, 8))
for column in cumulative_data.columns:
    plt.plot(cumulative_data.index, cumulative_data[column], label=column)

plt.xlabel('Order Date')
plt.ylabel('Cumulative Amount')
plt.title('Cumulative Amount Increase for Top 10 Items Over Time')
plt.legend(title='Item Details', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### Purchases of UPC Code Over Time with Trend Line

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates

# Convert Order_Date to a datetime format if not already
receipts_df['Order_Date'] = pd.to_datetime(receipts_df['Order_Date'])

# Filter for the specific UPC Code
specific_upc_df = receipts_df[receipts_df['UPC_Code'] == '2100065893']

# Group by Order_Date and sum the Amount
grouped_data = specific_upc_df.groupby('Order_Date')['Amount'].sum().reset_index()

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(grouped_data['Order_Date'], grouped_data['Amount'], marker='o', linestyle='-', label='Amount')

# Add labels for each data point
for i in range(len(grouped_data)):
    plt.text(grouped_data['Order_Date'][i], grouped_data['Amount'][i], f'{grouped_data["Amount"][i]:.2f}',
             ha='center', va='bottom')

# Fitting a linear trend line
z = np.polyfit(mdates.date2num(grouped_data['Order_Date']), grouped_data['Amount'], 1)
p = np.poly1d(z)
plt.plot(grouped_data['Order_Date'], p(mdates.date2num(grouped_data['Order_Date'])), "r--", label='Trend Line')

plt.xlabel('Order Date')
plt.ylabel('Amount')
plt.title('Purchases of UPC Code Over Time with Trend Line')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


### Total Savings Over Time

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=receipts_df, x='Order_Date', y='Total_Savings', color='#a29bfe')
plt.xlabel('Order Date')
plt.ylabel('Total Savings')
plt.title('Total Savings Over Time')
plt.show()
